# Trie Study

An enquiry into the [Trie](https://en.wikipedia.org/wiki/Trie) data structure, as well as its more space-optimised version, the [DAFSA or DAWG](https://en.wikipedia.org/wiki/Deterministic_acyclic_finite_state_automaton).  

![Trie](1024px-Trie_example.png)
Testing implementations proposals found here: https://stackoverflow.com/a/11016430.

In [19]:
def make_trie(*words, _end='_end'):
    root = dict()
    for word in words:
        current_dict = root
        for letter in word:
            current_dict = current_dict.setdefault(letter, {})
        current_dict[_end] = _end
    return root

In [28]:
make_trie('blah', 'diblah', 'blahsm', 'digamma', 'blahblahtor')

{'b': {'l': {'a': {'h': {'_end': '_end',
     's': {'m': {'_end': '_end'}},
     'b': {'l': {'a': {'h': {'t': {'o': {'r': {'_end': '_end'}}}}}}}}}}},
 'd': {'i': {'b': {'l': {'a': {'h': {'_end': '_end'}}}},
   'g': {'a': {'m': {'m': {'a': {'_end': '_end'}}}}}}}}

In [16]:
def make_trie_test(*words, _end='_end'):
    root = dict()
    for word in words:
        print('at word:', word)
        current_dict = root
        for letter in word:
            print('at letter:', letter)
            '''
            The .setdefault method adds both arguments if the key ('letter') is not in the dictionary
            *and returns* the second argument! (If it finds the key, it returns its value.) 
            That means that 'current_dict' will become {} at each iteration of the loop, the new level
            on which the new .setdict will perform its search, thereby creating 
            a recursive pattern (embedded dictionaries). As the returned object is part of current_dict,
            itself at first root (not deep copies of it), we are in fact filling root as we go along.
            '''
            current_dict = current_dict.setdefault(letter, {}) 
        current_dict[_end] = _end
        print(root)
        print('-'*40)
    return root

In [17]:
make_trie_test('cat', 'cats', 'catty')

at word: cat
at letter: c
at letter: a
at letter: t
{'c': {'a': {'t': {'_end': '_end'}}}}
----------------------------------------
at word: cats
at letter: c
at letter: a
at letter: t
at letter: s
{'c': {'a': {'t': {'_end': '_end', 's': {'_end': '_end'}}}}}
----------------------------------------
at word: catty
at letter: c
at letter: a
at letter: t
at letter: t
at letter: y
{'c': {'a': {'t': {'_end': '_end', 's': {'_end': '_end'}, 't': {'y': {'_end': '_end'}}}}}}
----------------------------------------


{'c': {'a': {'t': {'_end': '_end',
    's': {'_end': '_end'},
    't': {'y': {'_end': '_end'}}}}}}

A demo of the attribution mechanism, why `firstdict` stores the values even when `seconddict` is systematically reassigned to the latest {} (a trick possible because the dictionary returned by `setdefault` is not a deep copy, but a pointer to the existing (sub)dict in question, meaning that throughout our loop we keep modifying the dictionary we created first. 

In [18]:
firstdict = dict()
seconddict = firstdict

seconddict["reblah"] = "gggblah"
print(firstdict)
seconddict = seconddict.setdefault("blah", {})
print(firstdict)
seconddict = seconddict.setdefault("diblah", {})
print(firstdict)
seconddict = seconddict.setdefault("rediblah", {})
print(firstdict)
print()

{'reblah': 'gggblah'}
{'reblah': 'gggblah', 'blah': {}}
{'reblah': 'gggblah', 'blah': {'diblah': {}}}
{'reblah': 'gggblah', 'blah': {'diblah': {'rediblah': {}}}}



The testing function.

In [26]:
def in_trie(trie, word, _end='_end'):
    current_dict = trie
    for letter in word:
        if letter in current_dict:
            # move down one level (into nested dict)
            current_dict = current_dict[letter]
        else:
            return False
    else: # once at the end of our word, check if _end is present
        if _end in current_dict:
            return True
        else:
            return False

In [30]:
test = make_trie('blah', 'diblah', 'blahsm', 'digamma', 'blahblahtor')
print(in_trie(test,'bla'))
print(in_trie(test,'blah'))
print(in_trie(test, 'blahs'))

False
True
False


----

The more classic way of doing it, described on [Wikipedia](https://en.m.wikipedia.org/wiki/Trie).

In [49]:
class Node():
    def __init__(self):  
        '''
        Note that using dictionary for children (as in this implementation) 
        would not allow lexicographic sorting mentioned in the next section (Sorting),
        because ordinary dictionary would not preserve the order of the keys
        '''
        self.children = {}  # mapping from character ==> Node
        self.value = None
    
class Trie():
    
    def __init__(self):
        self.root = Node()
        
    def find(self, key):
        node = self.root
        for char in key:
            if char in node.children:
                node = node.children[char]
            else:
                print('not found')
                return None
        else:
            if node.value is not None: print('in trie!')
            else: print('not found')

    def insert(self, string, value='_end'):
        node = self.root
        i = 0
        while i < len(string):
            if string[i] in node.children:
                node = node.children[string[i]]
                i += 1
            else:
                break

        # append new nodes for the remaining characters, if any
        while i < len(string):
            node.children[string[i]] = Node() # create new node
            node = node.children[string[i]]   # switch current node to it
            i += 1

        # store value in the terminal node
        node.value = value

In [50]:
trie_test = Trie()
trie_test.insert("blah")
trie_test.find('blah')
trie_test.find('bla')

in trie!
not found


A list of professional implementations for further research, found [here](https://stackoverflow.com/a/21303098):

 - [marisa-trie][1] - a C++ based implementation.
 - [python-trie][2] - a simple pure python implementation.
 - [PyTrie][3] - a more advanced pure python implementation.
 - [pygtrie][4] - a pure python implementation by Google.
 - [datrie][5] - a double array trie implementation based on [libdatrie][6].


  [1]: https://pypi.python.org/pypi/marisa-trie/
  [2]: https://github.com/bdimmick/python-trie
  [3]: https://pypi.python.org/pypi/PyTrie
  [4]: https://github.com/google/pygtrie
  [5]: https://pypi.org/project/datrie/
  [6]: https://linux.thai.net/~thep/datrie/datrie.html